In [1]:
import copy
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import deeppavlov
from deeppavlov.dataset_readers.ontonotes_reader import OntonotesReader
from deeppavlov.models.preprocessors.capitalization import CapitalizationPreprocessor
from deeppavlov.models.embedders.glove_embedder import GloVeEmbedder

/home/kostrovskiy/prog/AI/NLP/DeepPavlov/env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
[nltk_data] Downloading package punkt to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/kostrovskiy/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2018-08-02 20:03:48.676 DEBU

In [2]:
reader = OntonotesReader()
dataset = reader.read(data_path='data/')
dataset['train'] = dataset['train'][:10]
dataset['test'] = dataset['train'][:10]
print(dataset.keys())
print('Num of train sentences: {}'.format(len(dataset['train'])))
print('Num of test sentences: {}'.format(len(dataset['test'])))

dict_keys(['train', 'valid', 'test'])
Num of train sentences: 10
Num of test sentences: 10


In [3]:
def split_tokens_tags(dataset: list):
    tokens = []
    tags = []
    for sample in dataset:
        tokens.append(sample[0])
        tags.append(sample[1])
    return tokens, tags

In [4]:
tokens_train,tags_train = split_tokens_tags(dataset['train'])
tokens_test,tags_test = split_tokens_tags(dataset['test'])
print(len(tokens_train))
print(len(tags_train))
print(len(tokens_test))
print(len(tags_test))

10
10
10
10


In [5]:
def tags2binary(tags, symb=True):
    tags = copy.deepcopy(tags)
    for seq in tags:
        for i in range(len(seq)):
            if symb:
                if seq[i] != 'O':
                    seq[i] = 'T'
            else:
                seq[i] = 1 if seq[i] != 'O' else 0
    return tags

tags_train = tags2binary(tags_train)
tags_test = tags2binary(tags_test)

In [6]:
class ElmoEmbedder():
    def __init__(self):
        self.elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
        sess = tf.Session()
        sess.run(tf.global_variables_initializer())
        self.sess = sess

    def get_tokens_embeddings(self, tokens_input: list, tokens_length:list=None):
        if not tokens_length:
            if isinstance(tokens_input[0], list):
                tokens_length = [len(seq) for seq in tokens_input]
            else:
                tokens_length = len(tokens_input)
        embeddings = self.elmo(
                        inputs={
                            "tokens": tokens_input,
                            "sequence_len": tokens_length
                        },
                        signature="tokens",
                        as_dict=True)["elmo"]
        embeddings = self.sess.run([embeddings])
        return embeddings[0]

In [7]:
def to_lower_case(tokens:list):
    tokens_lower = []
    for seq in tokens:
        tokens_lower.append([])
        for token in seq:
            tokens_lower[-1].append(token.lower())
#     print(tokens_lower)
    return tokens_lower

In [8]:
def get_tokens_len(tokens):
    if isinstance(tokens[0], str):
        tokens = [tokens]
    return [len(seq) for seq in tokens]

In [9]:
def add_padding(tokens:list):
    if isinstance(tokens[0], str):
        return tokens, len(tokens)
    elif isinstance(tokens[0], list):
        tokens = copy.deepcopy(tokens)
        max_len = 0
        for seq in tokens:
            if len(seq) > max_len:
                max_len = len(seq)
        for seq in tokens:
            i = len(seq)
            while i < max_len:
                seq.append('')
                i += 1
        return tokens
    else:
        raise Exception('tokens should be either list of strings or list of lists of strings')

In [10]:
def flatten_list(ar: list):
    flat = []
    for sublist in ar:
        flat += sublist
    return flat

In [11]:
def select_list_elements(ar: list, indices: list):
    return [ar[i] for i in indices]

In [12]:
class CompositeEmbedder():
    def __init__(self, use_elmo=True, elmo_scale=1., use_cap_feat=False, use_glove=False):
        self.use_elmo = use_elmo
        self.elmo_scale = elmo_scale
        self.use_cap_feat = use_cap_feat
        self.use_glove = use_glove
        if self.use_elmo:
            self.elmo = ElmoEmbedder()
        if self.use_cap_feat:
            self.cap_prep = CapitalizationPreprocessor()
        if self.use_glove:
            self.glove = GloVeEmbedder('embeddings/glove.6B/glove.6B.100d.txt', pad_zero=True)
        
    def embed(self, tokens: list):
        if isinstance(tokens[0], str):
            tokens = [tokens]
        # Get ELMo embeddings
        if self.use_elmo:
            tokens_input = add_padding(tokens)
            tokens_length = get_tokens_len(tokens)
            embeddings = self.elmo.get_tokens_embeddings(tokens_input, tokens_length)
            embeddings *= self.elmo_scale
            embed_size = embeddings.shape[-1]
#             print(embeddings.shape)
#             print(embed_size)

        # Use capitalization features
        if self.use_cap_feat:
#             print('Use capitalization features')
            cap_features = self.cap_prep(tokens)
    #         print(cap_features)
#             print(cap_features.shape)
            embeddings = np.concatenate((embeddings, cap_features), axis=2)
            embed_size = embeddings.shape[-1]
#             print(embeddings.shape)

        # Use GloVe embeddings
        if self.use_glove:
#             print('Use GloVe')
            
            glove_embed = self.glove(to_lower_case(tokens))
            glove_embed = np.array(glove_embed)
            if not self.use_elmo:
                embeddings = glove_embed
            else: 
                embeddings = np.concatenate((embeddings, glove_embed), axis=2)
            embed_size = embeddings.shape[-1]
#             print(embeddings.shape)
        
        return embeddings

In [13]:
def calc_sim(token_vec, ne_prototype)->dict:
    sim = {}
    sim['euc_dist'] = np.linalg.norm(token_vec - ne_prototype)
    sim['dot_prod'] = np.dot(token_vec, ne_prototype)
    sim['cosine'] = np.dot(token_vec, ne_prototype)/(np.linalg.norm(token_vec)*np.linalg.norm(ne_prototype)) if np.linalg.norm(ne_prototype) != 0 else 0
    return sim

In [14]:
def calc_sim_batch(tokens: list, embeddings: np.ndarray, ne_prototype: np.ndarray)->list:
    sim_list = []
    tokens_length = get_tokens_len(tokens)
    for i in range(len(tokens_length)):
        sim_list.append([])
        for j in range(tokens_length[i]):
            token_vec = embeddings[i,j,:]
            sim_list[i].append(calc_sim(token_vec, ne_prototype))
    return sim_list

In [15]:
def flatten_sim(sim_list):
    sims_flat = {'euc_dist': [], 'dot_prod': [], 'cosine': []}
    for i in range(len(sim_list)):
        for j in range(len(sim_list[i])):
            for sim_type in ['euc_dist', 'dot_prod', 'cosine']:
                sims_flat[sim_type].append(sim_list[i][j][sim_type])
    for sim_type in ['euc_dist', 'dot_prod', 'cosine']:
        sims_flat[sim_type] = np.array(sims_flat[sim_type])
    return sims_flat

In [16]:
def calc_ne_centroid_vec(tokens: list, tags: list, embeddings: np.ndarray=None, embedder: CompositeEmbedder=None):
#     print(tokens)
#     print(tags)
    no_ne_tags = False
    # Calculate embeddings
    if embedder != None:
        embeddings = embedder.embed(tokens)

    # Calculate average vector for ne-tags
    embed_size = embeddings.shape[-1]
    ne_prototype = np.zeros((embed_size,))
    tokens_length = get_tokens_len(tokens)
    n_ne_tags = 0
    for i in range(len(tokens_length)):
        for j in range(tokens_length[i]):
            if tags[i][j] == 'T':
                ne_prototype += embeddings[i,j,:].reshape((embed_size,))
                n_ne_tags += 1
    if n_ne_tags != 0:
        ne_prototype /= n_ne_tags
    else: 
        no_ne_tags = True
#     print('ne mean vector: {}'.format(ne_prototype))
    
    # Calculate similarities
    sim_list = calc_sim_batch(tokens, embeddings, ne_prototype)
    
    cosine_sim_flat = flatten_sim(sim_list)
    tags_flat = np.array(flatten_list(tags2binary(tags, symb=False)))
    best_threshold = 1
    if tags_flat[tags_flat == 0].size == 0:
        best_threshold = -1
    elif not no_ne_tags:
        sim_min_ne = np.min(cosine_sim_flat['cosine'][tags_flat == 1])
        sim_max_words = np.max(cosine_sim_flat['cosine'][tags_flat == 0])          
        best_threshold = np.mean([sim_min_ne, sim_max_words])
    
    del embeddings
    return ne_prototype, best_threshold

In [17]:
def infer_tags(tokens: len, ne_prototype: np.ndarray, sim_type='cosine', threshold=0.5, embeddings: np.ndarray=None, embedder: CompositeEmbedder=None):
    if isinstance(tokens[0], str):
        tokens = [tokens]
    
    tokens_length = get_tokens_len(tokens)
#     print(tokens_length)
    
    # Calculate embeddings
    if embedder != None:
        embeddings = embedder.embed(tokens)
    
    # Calculate similarities
    sim_list = calc_sim_batch(tokens, embeddings, ne_prototype)

    tags = [['T' if sim_list[i][j][sim_type] > threshold else 'O' for j in range(tokens_length[i])] for i in range(len(tokens_length))]
#     print(tags)
#     print(len(tags))
    return tags, sim_list

In [18]:
embedder = CompositeEmbedder(use_elmo=True, elmo_scale=1, use_cap_feat=True, use_glove=True)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


2018-08-02 20:04:05.623 INFO in 'tensorflow'['tf_logging'] at line 160: Using /tmp/tfhub_modules to cache modules.


INFO:tensorflow:Initialize variable module/aggregation/scaling:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/scaling


2018-08-02 20:04:06.232 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/aggregation/scaling:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/scaling


INFO:tensorflow:Initialize variable module/aggregation/weights:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with aggregation/weights


...

2018-08-02 20:04:06.549 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/kernel:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/kernel


INFO:tensorflow:Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel


2018-08-02 20:04:06.557 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel


INFO:tensorflow:Initialize variable module/bilm/char_embed:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/char_embed


2018-08-02 20:04:06.565 INFO in 'tensorflow'['tf_logging'] at line 116: Initialize variable module/bilm/char_embed:0 from checkpoint b'/tmp/tfhub_modules/9bb74bc86f9caffc8c47dd7b33ec4bb354d9602d/variables/variables' with bilm/char_embed
2018-08-02 20:04:07.966 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 60: [loading embeddings from `/home/kostrovskiy/prog/AI/NLP/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt`]
2018-08-02 20:04:07.966 INFO in 'gensim.models.keyedvectors'['keyedvectors'] at line 204: loading projection weights from /home/kostrovskiy/prog/AI/NLP/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt
2018-08-02 20:04:07.967 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': '/home/kostrovskiy/prog/AI/NLP/DeepPavlov/download/embeddings/glove.6B/glove.6B.100d.txt'}
2018-08-02 20:04:40.749 INFO in 'gensim.models.keyedvectors'['keyedvectors'] at line 266: loaded (400000, 100) matr

In [19]:
embeddings_test = embedder.embed(tokens_test)
print(embeddings_test.shape)

(10, 37, 1128)


### Calculate statistics

In [20]:
class SubSamplesGenerator():
    def __init__(self, total_samples, n_subsamples):
        self.total_samples = total_samples
        self.n_subsamples = n_subsamples
        self.indices = np.arange(1, n_subsamples+1)
        self.has_next_flag = True
        
    def has_next(self):
        return self.has_next_flag
    
    def get_next(self):
        indices = self.indices.copy()
        self.update() 
        return indices - 1
    
    def update(self):
        pos = self.n_subsamples
        stop = False
        while True:
            self.indices[pos-1] += 1
            if self.indices[pos-1] > self.total_samples - self.n_subsamples + pos:
                if pos == 1:
                    self.has_next_flag = False
                    break
                pos -= 1
            else:
                for i in range(pos, self.n_subsamples):
                    self.indices[i] = self.indices[i-1] + 1
                break

In [21]:
# gen = SubSamplesGenerator(7,4)
# subsamples = []
# while gen.has_next():
#     subsamples.append(gen.get_next())
# print(subsamples)

In [22]:
from sklearn.metrics import f1_score

In [ ]:
f1_score_list = []
for n_examples in range(1,11):
    print('~~~~~~~~ {}-shot case ~~~~~~~~'.format(n_examples))
    gen = SubSamplesGenerator(len(tokens_train),n_examples)
    while gen.has_next():
        f1_score_list.append([])
        sample_indices = gen.get_next()
        print(sample_indices)
        tokens_train_sample = select_list_elements(tokens_train, sample_indices)
        tags_train_sample = select_list_elements(tags_train, sample_indices)
#         embeddings_train_sample = embeddings_train[sample_indices]
        ne_prototype, threshold_train = calc_ne_centroid_vec(tokens_train_sample, tags_train_sample, embedder=embedder)
        tags_test_pred, sim_list_test = infer_tags(tokens_test, ne_prototype, sim_type='cosine', threshold=threshold_train, embeddings=embeddings_test)
        target = flatten_list(tags2binary(tags_test, symb=False))
        pred = flatten_list(tags2binary(tags_test_pred, symb=False))
#         print(len(target))
#         print(len(pred))
        f1 = f1_score(target, pred)
        f1_score_list[-1].append(f1)
print(f1_score_list)

~~~~~~~~ 1-shot case ~~~~~~~~
[0]


/home/kostrovskiy/prog/AI/NLP/DeepPavlov/env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


[1]
[2]
[3]
[4]
[5]
[6]
[7]
[8]
[9]
~~~~~~~~ 2-shot case ~~~~~~~~
[0 1]
[0 2]
[0 3]
[0 4]
[0 5]
[0 6]
[0 7]
[0 8]
[0 9]
[1 2]
[1 3]
[1 4]
[1 5]
[1 6]
[1 7]
[1 8]
[1 9]
[2 3]
[2 4]
[2 5]
[2 6]
[2 7]
[2 8]
[2 9]
[3 4]
[3 5]
[3 6]
[3 7]
[3 8]
[3 9]
[4 5]
[4 6]
[4 7]
[4 8]
[4 9]
[5 6]
[5 7]
[5 8]
[5 9]
[6 7]
[6 8]
[6 9]
[7 8]
[7 9]
[8 9]
~~~~~~~~ 3-shot case ~~~~~~~~
[0 1 2]
[0 1 3]
[0 1 4]
[0 1 5]
[0 1 6]
[0 1 7]
[0 1 8]
[0 1 9]
[0 2 3]
[0 2 4]
[0 2 5]
[0 2 6]
[0 2 7]
[0 2 8]
[0 2 9]
[0 3 4]
[0 3 5]
[0 3 6]
[0 3 7]
[0 3 8]
[0 3 9]
[0 4 5]
[0 4 6]
[0 4 7]
[0 4 8]
[0 4 9]
[0 5 6]
[0 5 7]
[0 5 8]
[0 5 9]
[0 6 7]
[0 6 8]
[0 6 9]
[0 7 8]
[0 7 9]
[0 8 9]
[1 2 3]
[1 2 4]
[1 2 5]
[1 2 6]
